In [339]:
import numpy as np
import os
import matplotlib
from matplotlib import pyplot as plt
from scipy import signal
from operator import attrgetter
from functools import reduce

GRAY_CONVERTER = np.array([0.2989, 0.5870, 0.1140])
EPSILON = np.finfo(float).eps

train_path = r'.\Group_1\train'
test_path = r'.\Group_1\test'


In [340]:
def relu(x):
    return np.where(x >= 0, x, 0)


def sigmoid(x):
    return np.apply_along_axis(lambda a: 1 / (1 + np.exp(-a)), 0, x)


def convert_to_gray(image):
    return np.dot(image[..., :3], GRAY_CONVERTER)


def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value


In [341]:
class ConvolutionLayer:
    def __init__(self, H2=2, W2=2, D2=1, K=1):
        self.H2 = H2
        self.W2 = W2
        self.D2 = D2
        self.K = K
        self.filters = np.random.normal(scale=0.3, size=(K, H2, W2, D2))
        self.bias = np.random.normal(scale=0.3, size=(K))

    def convolve(self, input):
        H1, W1, _ = input.shape

        H2, W2, D2 = attrgetter('H2', 'W2', 'D2')(self)

        for i in range(H1 - H2 + 1):
            for j in range(W1 - W2 + 1):
                section = input[i: i + H2, j: j + W2]
                yield section, i, j

    def calculate(self, input):
        H1, W1, _ = input.shape

        self.last_input = input

        H2, W2, D2, K, filters, bias = attrgetter(
            'H2', 'W2', 'D2', 'K', 'filters', 'bias')(self)

        feature_maps = np.zeros(((H1 - H2 + 1), (W1 - W2 + 1), K))

        for section, i, j in self.convolve(input):
            feature_maps[i, j] = bias

            if D2 == 1:
                feature_maps[i, j] += np.sum(section * filters, axis=(1, 2))
                continue

            for k in range(D2):
                feature_maps[i, j] += np.sum(section[:, :, k]
                                             * filters[:, :, :, k], axis=(1, 2))

        return feature_maps

    def backprop(self, d_L_d_out, eta):
        
        H2, W2, D2, K, filters, bias = attrgetter(
            'H2', 'W2', 'D2', 'K', 'filters', 'bias')(self)


        d_L_d_filters = np.zeros(self.filters.shape)
        d_L_d_input = np.zeros(self.last_input.shape)

        for section, i, j in self.convolve(self.last_input):
            for f in range(self.K):
                for k in range(self.D2):
                    d_L_d_filters[f, :, :, k] += d_L_d_out[i, j, f] * section[:, :, k]
                    d_L_d_input[i : i + H2, j : j + W2, k] += d_L_d_out[i, j, f] * filters[f, :, :, k]
        self.filters -= eta * d_L_d_filters
        self.bias -= eta * np.sum(d_L_d_out, axis=(0, 1))

        return d_L_d_input

In [342]:
class PoolingLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def pooling(self, input):
        H, W, _ = input.shape

        for i in range(H // self.pool_size):
            for j in range(W // self.pool_size):
                x, y = i * self.pool_size, j * self.pool_size
                section = input[x: x + self.pool_size, y:y + self.pool_size]
                yield section, i, j

    def calculate(self, input):
        H, W, K = input.shape
        self.last_input = input
        pooled_maps = np.zeros((H // self.pool_size, W // self.pool_size, K))

        for section, i, j in self.pooling(input):
            pooled_maps[i, j] = np.amax(section, axis=(0, 1))

        return pooled_maps

    def backprop(self, d_L_d_out):
        
        d_L_d_input = np.zeros(self.last_input.shape)

        for section, i, j in self.pooling(self.last_input):
            h, w, d = section.shape
            amax = np.amax(section, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for k2 in range(d):
                        if section[i2, j2, k2] == amax[k2]:
                            d_L_d_input[i * self.pool_size + i2, j * self.pool_size +  + j2, k2] = d_L_d_out[i, j, k2]
            
        
        return d_L_d_input

In [368]:
class SoftmaxLayer:
    def __init__(self, n_classes, input_size):
        self.n_classes = n_classes
        self.input_size = input_size
        self.w = np.random.normal(scale=0.3, size=(input_size, n_classes))
        self.b = np.random.normal(scale=0.3, size=(n_classes, ))

    def calculate(self, input):
        self.last_input_shape = input.shape

        flattened_input = input.flatten()
        self.last_input = flattened_input
        input_size, n_classes = attrgetter('input_size', 'n_classes')(self)

        output = np.dot(flattened_input, self.w) + self.b
        output = output - np.amax(output)
        self.last_outputs = output
        return np.exp(output) / np.sum(np.exp(output), axis=0)

    def backprop(self, d_L_d_out, eta):
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue
            t_exp = np.exp(self.last_outputs)
            S = np.sum(t_exp)

            d_out_d_t = (-t_exp[i] * t_exp) / (S ** 2)
            d_out_d_t[i] = (t_exp[i] * (S - t_exp[i])) / (S ** 2)

            print(t_exp)
            print(d_out_d_t)
            
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_input = self.w

            d_L_d_t = gradient * d_out_d_t

            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_input = d_t_d_input @ d_L_d_t

            self.w -= eta * d_L_d_w
            self.b -= eta * d_L_d_b

            return d_L_d_input.reshape(self.last_input_shape)

        




In [369]:
conv1_layer = ConvolutionLayer(3, 3, 3, 32)
conv2_layer = ConvolutionLayer(3, 3, 32, 64)
pool_layer = PoolingLayer(2)
# softmax_layer = SoftmaxLayer(3, 15 * 15 * 32)
softmax_layer = SoftmaxLayer(3, 14 * 14 * 64)

In [370]:
image = matplotlib.image.imread(os.path.join(train_path, 'bird', '0000.jpg'))


In [371]:
def apply(image, label):
    out = relu(conv1_layer.calculate(image))
    out = relu(conv2_layer.calculate(out))
    out = pool_layer.calculate(out)
    out = softmax_layer.calculate(out)

    loss = -np.log(out[label] + EPSILON)
    acc = 1 if np.argmax(out) == label else 0

    return out, loss, acc


def train(image, label, eta=0.005):
    out, loss, acc = apply(image, label)

    gradient = np.zeros(3)
    gradient[label] = -1 / (out[label] + EPSILON)
    print(gradient)
    gradient = softmax_layer.backprop(gradient, eta)
    # print(gradient)
    # gradient = pool_layer.backprop(gradient)
    # gradient = conv2_layer.backprop(gradient, eta)
    # gradient = conv1_layer.backprop(gradient, eta)

    return loss, acc


In [327]:
images = []
for label, item in enumerate(['bird', 'deer', 'truck']):  
    path = os.path.join(train_path, item)
    images += map(lambda img: (matplotlib.image.imread(os.path.join(path, img)), label),os.listdir(path))

In [328]:
import random

random.shuffle(images)

In [ ]:
true_val, total_loss = 0, 0


for i, (img, label) in enumerate(images):
    loss, acc = train(img, label, 0.01)
    true_val += acc
    total_loss += loss
    print('[Step %d]: Average Loss %.3f | Accuracy: %d%%' %
            (i + 1, total_loss / (i + 1), true_val * 100 / (i + 1)))
        

In [ ]:
test_images = []
for label, item in enumerate(['bird', 'deer', 'truck']):  
    path = os.path.join(test_path, item)
    test_images += map(lambda img: (matplotlib.image.imread(os.path.join(path, img)), label),os.listdir(path))

random.shuffle(test_images)

In [ ]:
true_val, total_loss = 0, 0


for i, (img, label) in enumerate(test_images):
    loss, acc = train(img, label, 0.01)
    true_val += acc
    total_loss += loss
    print('[Step %d]: Average Loss %.3f | Accuracy: %d%%' %
            (i + 1, total_loss / (i + 1), true_val * 100 / (i + 1)))
        

In [372]:
train(image, 0)

[-4.50359963e+15  0.00000000e+00  0.00000000e+00]
[0. 0. 1.]
[ 0. -0. -0.]


(36.04365338911715, 0)